In [4]:
from pathlib import Path
import torch
torch.set_float32_matmul_precision('medium')
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import numpy as np
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from dataset import CardiacDataset
from model import UNet

In [6]:
seq = iaa.Sequential([
    iaa.Affine(scale=(0.85, 1.15),
               rotate = (-45,45)),
    iaa.ElasticTransformation()
])

In [7]:
train_path = Path("Preprocessed/train/")
val_path  = Path("Preprocessed/val/")

train_dataset = CardiacDataset(train_path, seq)
val_dataset = CardiacDataset(val_path, None)

In [8]:
batch_size = 8
num_workers = 4

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, persistent_workers=True, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, persistent_workers=True, shuffle=False)

In [9]:
class DiceLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred, mask):
        pred = torch.flatten(pred)
        mask = torch.flatten(mask)

        counter = (pred * mask).sum()
        denum = pred.sum() + mask.sum()  + 1e-8
        dice = (2*counter) / denum
        return 1 - dice 


In [10]:
from typing import Any


class AtriumSegmentation(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = UNet()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = 1e-4)
        self.loss_fn = DiceLoss()

    def forward(self, data):
        return torch.sigmoid(self.model(data))  
    
    def training_step(self, batch, batch_idx):
        mri, mask = batch
        mask = mask.float()
        pred = self(mri)

        loss = self.loss_fn(pred, mask)
        self.log("Train Dice", loss)
        if batch_idx % 50 ==0:
            self.log_images(mri.cpu(), pred.cpu(), mask.cpu(), "Train")
        return loss
    
    def validation_step(self, batch, batch_idx):
        mri, mask = batch
        mask = mask.float()
        pred = self(mri)

        loss = self.loss_fn(pred, mask)
        self.log("Val Dice", loss)
        if batch_idx % 50 ==0:
            self.log_images(mri.cpu(), pred.cpu(), mask.cpu(), "Val")
        return loss
    
    def log_images(self, mri, pred, mask, name):
        pred = pred > 0.5

        fig, axis = plt.subplots(1,2)
        axis[0].imshow(mri[0][0], cmap = "bone")
        mask_ = np.ma.masked_where(mask[0][0]==0, mask[0][0])
        axis[0].imshow(mask_, alpha = 0.6)

        axis[1].imshow(mri[0][0], cmap = "bone")
        mask_ = np.ma.masked_where(mask[0][0]==0, mask[0][0])
        axis[1].imshow(mask_, alpha = 0.6)

        self.logger.experiment.add_figure(name, fig, self.global_step)

    def configure_optimizers(self):
        return [self.optimizer]
    

In [7]:
torch.manual_seed(0)
model = AtriumSegmentation()

In [8]:
checkpoint_callback = ModelCheckpoint(
    monitor="Val Dice",
    save_top_k=10,
    mode ='min'
    )

In [9]:
trainer = pl.Trainer(devices =1, logger =TensorBoardLogger(save_dir="logs"), log_every_n_steps=1, callbacks=checkpoint_callback, max_epochs=30)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model, train_loader, val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3050 6GB Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: logs\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type     | Params | Mode 
---------------------------------------------
0 | model   | UNet     | 7.8 M  | train
1 | loss_fn | DiceLoss | 0      | train
---------------------------------------------
7.8 M     Trainable params
0         Non-trainable params
7.8 M     Total params
31.127    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


In [2]:
import nibabel as nib
from  tqdm.notebook import tqdm
from celluloid import Camera

In [11]:
model = AtriumSegmentation.load_from_checkpoint("weights/70.ckpt")

C:\Users\lehar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_lightning\utilities\migration\migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
Lightning automatically upgraded your loaded checkpoint from v1.3.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint d:\Machine_Learning\pytorch\py_torch for medical_imaging_course\06-Atrium-Segmentation\06-Atrium-Segmentation\weights\70.ckpt`


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
model.eval();
model.to(device)

AtriumSegmentation(
  (model): UNet(
    (layer1): DoubleConv(
      (step): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
    )
    (layer2): DoubleConv(
      (step): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
    )
    (layer3): DoubleConv(
      (step): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
    )
    (layer4): DoubleConv(
      (step): Sequential(
        (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2):

In [15]:
preds = []
labels = []

for slice, label in tqdm(val_loader):
    slice = torch.tensor(slice).to(device)
    with torch.no_grad():
        pred = model(slice)
    preds.append(pred.detach().cpu().numpy())
    labels.append(label)
preds = np.concatenate(preds)
labels = np.concatenate(labels)



  0%|          | 0/43 [00:00<?, ?it/s]

C:\Users\lehar\AppData\Local\Temp\ipykernel_620\3575032327.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  slice = torch.tensor(slice).to(device)


In [16]:
1 - model.loss_fn(torch.from_numpy(preds), torch.from_numpy(labels))

tensor(0.9539)